In [4]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from SystemGeneral.system import System
from utils.utils import generate_series
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
import logging
from tqdm import tqdm
rpy2_logger.setLevel(logging.ERROR)   # will display errors, but not warnings

warnings.filterwarnings('ignore')

In [5]:
Q = 301
N = 800

In [6]:
column_names = ['\omega_1', '\omega_2', '\delta_min', '\sigma', 'B', 'T', 'L', 'FPR', 'TPR', 'FNR']
results = pd.DataFrame(columns=column_names)
# results.to_latex(escape=False)

In [7]:
%%time

w1 = 1/10
w1_str = f"1/10"
for denom in range(3, 10):
    w2 = 1/denom
    w2_str = f"1/{denom}"
    sigma = 0.5

    delta_min = 1/50
    k = 30
    iter_num = 200

    np.random.seed(1234567)
    for sigma in np.arange(0, 0.8, 0.1):
        fp = 0
        fn = 0
        tp = 0
        for i in tqdm(range(iter_num), desc=f"Моделирование для sigma = {sigma}"):
            series = generate_series(w1, w2, Q, N) + np.random.normal(scale=sigma, size=N)
            system = System(series, k, delta_min)
            Q_hat = system()
            if Q_hat - Q < 0:
                fp += 1
            if Q_hat - Q > k:
                fn += 1
            if Q_hat - Q <= k:
                tp += 1
        fpr = fp / iter_num
        fnr = fn / iter_num
        tpr = tp / iter_num
        results = results.append(dict(zip(column_names, [w1, w2, delta_min, sigma, system.B, system.T, system.L, fpr, tpr, fnr])), ignore_index=True)

Моделирование для sigma = 0.7000000000000001: 100%|██████████| 200/200 [06:12<00:00,  1.86s/it]

CPU times: total: 48min 17s
Wall time: 48min 18s


In [8]:
system.B, system.T, system.L

(133, 79, 71)

In [9]:
results

,\omega_1,\omega_2,\delta_min,\sigma,B,T,L,FPR,TPR,FNR
0,0.1,0.2,0.02,0.0,133.0,79.0,71.0,0.0,1.0,0.0
1,0.1,0.2,0.02,0.1,133.0,79.0,71.0,0.0,1.0,0.0
2,0.1,0.2,0.02,0.2,133.0,79.0,71.0,0.0,1.0,0.0
3,0.1,0.2,0.02,0.3,133.0,79.0,71.0,0.0,1.0,0.0
4,0.1,0.2,0.02,0.4,133.0,79.0,71.0,0.0,1.0,0.0
5,0.1,0.2,0.02,0.5,133.0,79.0,71.0,0.0,0.97,0.03
6,0.1,0.2,0.02,0.6,133.0,79.0,71.0,0.01,0.94,0.06
7,0.1,0.2,0.02,0.7,133.0,79.0,71.0,0.025,0.92,0.08


In [6]:
# results

,\omega_1,\omega_2,\delta_min,\sigma,B,T,L,FPR,TPR,FNR
0,0.1,0.2,0.02,0.0,133.0,133.0,119.0,0.0,1.0,0.0
1,0.1,0.2,0.02,0.1,133.0,133.0,119.0,0.0,1.0,0.0
2,0.1,0.2,0.02,0.2,133.0,133.0,119.0,0.0,1.0,0.0
3,0.1,0.2,0.02,0.3,133.0,133.0,119.0,0.0,1.0,0.0
4,0.1,0.2,0.02,0.4,133.0,133.0,119.0,0.0,0.935,0.065
5,0.1,0.2,0.02,0.5,133.0,133.0,119.0,0.0,0.845,0.155
6,0.1,0.2,0.02,0.6,133.0,133.0,119.0,0.035,0.815,0.185
7,0.1,0.2,0.02,0.7,133.0,133.0,119.0,0.115,0.735,0.265


In [6]:
# results['B'] = [system.B] * len(results)
# results['T'] = [system.T] * len(results)
# results['L'] = [system.L] * len(results)

In [7]:
# results

,\omega_1,\omega_2,\delta_min,\sigma,B,T,L,FPR,TPR,FNR
0,0.1,0.2,0.02,0.0,133.0,133.0,106.0,0.0,1.0,0.0
1,0.1,0.2,0.02,0.1,133.0,133.0,106.0,0.0,1.0,0.0
2,0.1,0.2,0.02,0.2,133.0,133.0,106.0,0.0,0.99,0.01
3,0.1,0.2,0.02,0.3,133.0,133.0,106.0,0.0,0.92,0.08
4,0.1,0.2,0.02,0.4,133.0,133.0,106.0,0.0,0.77,0.23
5,0.1,0.2,0.02,0.5,133.0,133.0,106.0,0.0,0.62,0.38
6,0.1,0.2,0.02,0.6,133.0,133.0,106.0,0.03,0.65,0.35
7,0.1,0.2,0.02,0.7,133.0,133.0,106.0,0.095,0.6,0.4


In [8]:
# results.to_csv('results.csv', index=False)

In [7]:
pd.read_csv('results.csv')

,\omega_1,\omega_2,\delta_min,\sigma,FPR,TPR,FNR,B,T,L
0,0.1,0.2,0.02,0.0,0.000,1.000,0.000,100,100,95
1,0.1,0.2,0.02,0.1,0.000,1.000,0.000,100,100,95
2,0.1,0.2,0.02,0.2,0.000,1.000,0.000,100,100,95
3,0.1,0.2,0.02,0.3,0.000,1.000,0.000,100,100,95
4,0.1,0.2,0.02,0.4,0.000,0.985,0.015,100,100,95
5,0.1,0.2,0.02,0.5,0.000,0.935,0.065,100,100,95
6,0.1,0.2,0.02,0.6,0.035,0.865,0.135,100,100,95
7,0.1,0.2,0.02,0.7,0.145,0.805,0.195,100,100,95
